In [1]:
!pip install langchain faiss-cpu PyMuPDF python-docx langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.1 MB/s eta 0:00:00


In [15]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
from google.colab import files
import fitz  # PyMuPDF
import io
import os
from docx import Document as DocxDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains.question_answering import load_qa_chain
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

2: Functions for Extracting Text from PDF Files

In [4]:
# Function to extract text from .pdf files
def extract_text_from_pdf(pdf_bytes):
    pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")
    text = "".join(page.get_text() for page in pdf_document)
    return text.replace("\n", "")


In [5]:
# Function to extract text from .docx files
def extract_text_from_docx(docx_bytes):
    docx_document = DocxDocument(io.BytesIO(docx_bytes))
    text = "".join(para.text for para in docx_document.paragraphs)
    return text.replace("\n", "")


In [6]:
# Function to extract text from .txt files
def extract_text_from_txt(txt_bytes):
    return txt_bytes.decode('utf-8').replace("\n", "")


In [7]:
# Function to upload documents
def upload_documents():
    uploaded = files.upload()
    documents = {filename: uploaded[filename] for filename in uploaded.keys() if filename.split('.')[-1].lower() in ['pdf', 'docx', 'txt']}
    return documents

# Get text from uploaded documents
documents = upload_documents()
texts = {filename: (extract_text_from_pdf(file_bytes) if filename.endswith('.pdf') else
                    extract_text_from_docx(file_bytes) if filename.endswith('.docx') else
                    extract_text_from_txt(file_bytes))
         for filename, file_bytes in documents.items()}


Saving 08_science_key_notes_ch_05_coal_and_petroleum.pdf to 08_science_key_notes_ch_05_coal_and_petroleum.pdf
Saving 08_science_key_notes_ch_06_combustion_and_flame.pdf to 08_science_key_notes_ch_06_combustion_and_flame.pdf
Saving Harsh_Resume_DS (4).pdf to Harsh_Resume_DS (4).pdf
Saving mugli.txt to mugli.txt
Saving qna_PDF.docx to qna_PDF.docx
Saving text_file.txt to text_file.txt


In [8]:
# Print the extracted texts
for filename, text in texts.items():
     print(f"Contents of {filename}:")
     print(text)

Contents of 08_science_key_notes_ch_05_coal_and_petroleum.pdf:
Material Downloaded From http://www.ncrtsolutions.inwww.ncrtsolutions.inKey Notes Portal for CBSE Notes, Test Papers, Sample Papers, Tips and Tricks Chapter – 5 Coal and Petroleum  • Natural Resources: Resources include everything provide by the nature. They form the wealth of a country. • Types of Natural Resources: (i)Inexhaustible: There are some resources that are present unlimited in nature and will not be exhausted even if used continuously. Example: Sunlight, Air. (ii)Exhaustible: These resources are limited and can soon get exhausted because of their excessive use. Example: Forests, wildlife, minerals, coal, petroleum, etc. •Coal, petroleum and natural gas are fossil fuels. •Fossil fuels were formed from the dead remains of living organisms millions of years ago. •Fossil fuels are exhaustible resources. •Coke, coal tar and coal gas are the products of coal. •Petroleum gas, petrol, diesel, kerosene, paraffin wax, lub

In [9]:
# Split text into chunks
chunk_size = 400  # Increased chunk size for larger documents
chunk_overlap = 100  # Moderate overlap to maintain context

splitter = RecursiveCharacterTextSplitter(
    length_function = len,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

chunks = [(doc_name, chunk) for doc_name, text in texts.items() for chunk in splitter.split_text(text)]
docs = [chunk[1] for chunk in chunks]

In [10]:
print(type(chunks),len(chunks))

<class 'list'> 57


In [11]:
print(type(docs),len(docs),docs)

<class 'list'> 57 ['Material Downloaded From http://www.ncrtsolutions.inwww.ncrtsolutions.inKey Notes Portal for CBSE Notes, Test Papers, Sample Papers, Tips and Tricks Chapter – 5 Coal and Petroleum  • Natural Resources: Resources include everything provide by the nature. They form the wealth of a country. • Types of Natural Resources: (i)Inexhaustible: There are some resources that are present unlimited in nature', 'Natural Resources: (i)Inexhaustible: There are some resources that are present unlimited in nature and will not be exhausted even if used continuously. Example: Sunlight, Air. (ii)Exhaustible: These resources are limited and can soon get exhausted because of their excessive use. Example: Forests, wildlife, minerals, coal, petroleum, etc. •Coal, petroleum and natural gas are fossil fuels. •Fossil', 'minerals, coal, petroleum, etc. •Coal, petroleum and natural gas are fossil fuels. •Fossil fuels were formed from the dead remains of living organisms millions of years ago. •F

In [16]:
def get_vector_store(text_chunks):
    # Initialize Hugging Face Embeddings
    model_name = "sentence-transformers/all-mpnet-base-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    return vector_store

# Example usage:
text_chunks = docs
vector_store = get_vector_store(text_chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
vector_store

In [19]:
def answer_question(question, vector_store):
    # Get the top 5 similar documents with their scores
    similar_docs_with_scores = vector_store.similarity_search_with_score(question, k=5)

    # Extract the top document and its score
    top_doc, score = similar_docs_with_scores[0]
    return  top_doc

In [21]:
question = "tell me two types ofNatural Resources:"
answers = answer_question(question, vector_store)
answers

Document(page_content='Material Downloaded From http://www.ncrtsolutions.inwww.ncrtsolutions.inKey Notes Portal for CBSE Notes, Test Papers, Sample Papers, Tips and Tricks Chapter – 5 Coal and Petroleum  • Natural Resources: Resources include everything provide by the nature. They form the wealth of a country. • Types of Natural Resources: (i)Inexhaustible: There are some resources that are present unlimited in nature')